# E-commerce Sales - Data Wrangling

In [1]:
#import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
%matplotlib inline

For this project we have 2 datasets: Amazon sales and International sales. Let's start our data wrangling with amazon sales data. First we need to upload it and check the data quality.

## Amazon Sales

In [2]:
amazon_df=pd.read_csv('Original datasets/Amazon Sale Report.csv', dtype={'Unnamed: 22':'object'})
amazon_df.head()

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [3]:
amazon_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount              121180 non-nul

First we'll drop the 'index' column as it repeats the indices of the dataframe and change the 'Date' column's data type to datetime.

In [4]:
amazon_df.drop('index', axis=1, inplace=True)
amazon_df['Date']=pd.to_datetime(amazon_df['Date'])

Now, let's create a new dataframe showing the number of unique values and null values in every column.

In [5]:
vals=pd.DataFrame(index=amazon_df.columns.values, data={'data_type':amazon_df.dtypes, 'unique_values':amazon_df.nunique(), \
                                                        'null_values':amazon_df.isnull().sum()})
print(vals.sort_values(['null_values', 'unique_values'], ascending=False))

                         data_type  unique_values  null_values
fulfilled-by                object              1        89698
promotion-ids               object           5787        49153
Unnamed: 22                 object              1        49050
Amount                     float64           1410         7795
currency                    object              1         7795
Courier Status              object              3         6872
ship-postal-code           float64           9459           33
ship-city                   object           8955           33
ship-state                  object             69           33
ship-country                object              1           33
Order ID                    object         120378            0
SKU                         object           7195            0
ASIN                        object           7190            0
Style                       object           1377            0
Date                datetime64[ns]             91      

So, for the 'ship_country' and 'currency' columns we have 1 unique value for each of them(India for country and Indian rupee for currency) and some missing values as well. </br>
And we have the same amount of missing values for the 'currency' and 'Amount' columns which probably means that the same transactions have both missing currency and amount. We'll check this statement with the code below.

In [6]:
print(amazon_df[amazon_df['currency'].isna()]['Amount'].value_counts(dropna=False))

NaN    7795
Name: Amount, dtype: int64


Our statement was true, we'll deal with the missing values in 'Amount' column later. </br>
Next we have 'Unnamed: 22' column which doesn't contain any valuable information: it has 1 unique value which is 'False' and about 50k missing values. We'll drop this column with 'ship-country' and 'currency' columns. </br>
The next column is the 'fulfilled-by' column which has the most amount of missing values - almost 90k, but we also have 'Fulfilment' column which probably shows the same information as the previous one. Let's check the unique values in both of them.

In [7]:
print(amazon_df[amazon_df['fulfilled-by'].isna()]['Fulfilment'].value_counts(dropna=False))

Amazon    89698
Name: Fulfilment, dtype: int64


In [8]:
print(amazon_df[['Fulfilment', 'fulfilled-by']].value_counts(dropna=False))

Fulfilment  fulfilled-by
Amazon      NaN             89698
Merchant    Easy Ship       39277
dtype: int64


So, all of the orders were fulfilled by 2 companies: Amazon and Easy ship. We can drop the 'fulfilled-by' column and replace the 'Merchant' values in 'Fulfilment' with 'Easy ship'.

In [9]:
amazon_df.drop(columns=['Unnamed: 22', 'ship-country', 'currency', 'fulfilled-by'], axis=1, inplace=True)

amazon_df['Fulfilment'].replace('Merchant', 'Easy Ship', inplace=True)
amazon_df.rename(columns={'Fulfilment':'Fulfillment'}, inplace=True)

Next we have 'promotion-ids' column which has almost 50k missing values which probably means absense of a promotion.We can fill the null values with 'no promotion' for now. Later in the project maybe we'll convert it into a boolean column showing if a promotion was used for the order or not.

In [10]:
amazon_df['promotion-ids'].fillna('no promotion', inplace=True)

We also have 33 missing values for 'ship-postal-code', 'ship-city' and 'ship-state' columns which will be dropped after ensuring that the rows with missing values are the same.

In [11]:
print(amazon_df[amazon_df['ship-postal-code'].isna()][['ship-city', 'ship-state']].value_counts(dropna=False))

ship-city  ship-state
NaN        NaN           33
dtype: int64


In [12]:
ship_null=amazon_df[amazon_df['ship-postal-code'].isna()]
amazon_df.drop(index=ship_null.index, inplace=True)

Next we have 'Courier Status' column. Let's take a look at its values.

In [13]:
amazon_df['Courier Status'].value_counts(dropna=False)

Shipped      109461
NaN            6869
Unshipped      6679
Cancelled      5933
Name: Courier Status, dtype: int64

We also have 'Status' column which provides the same information but have 13 unique values without null values. Let's take a look how values in those 2 columns are connected to each other.

In [14]:
status_gr=amazon_df.groupby('Courier Status', dropna=False)['Status']
status_gr.value_counts()

Courier Status  Status                       
Cancelled       Cancelled                         5838
                Shipped                             93
                Pending                              2
Shipped         Shipped                          77580
                Shipped - Delivered to Buyer     28754
                Shipped - Returned to Seller      1947
                Shipped - Picked Up                973
                Shipped - Returning to Seller      145
                Shipped - Out for Delivery          35
                Shipped - Rejected by Buyer         11
                Pending                             10
                Shipped - Lost in Transit            5
                Shipped - Damaged                    1
Unshipped       Cancelled                         5629
                Pending                            646
                Pending - Waiting for Pick Up      281
                Shipped                            115
                Shi

As we can see, most of the missing values have 'Cancelled' status. So, we'll fill the missing values with 'Cancelled' value.

In [15]:
amazon_df['Courier Status'].fillna('Cancelled', inplace=True)

And the last column with the missing values is the 'Amount' column which represents the total price of each order. To fill the missing values we'll use the SKU code of the item to fill the price if the item was sold more than once. In order to do that we'll create a separate dataset with the SKU codes and the prices for each of them. But first let's check if the amount column for each SKU is unique.

In [16]:
sku_prices=amazon_df.groupby('SKU')[['Amount']].nunique().sort_values('Amount', ascending=False)
sku_prices.head()

,Amount
SKU,
JNE3797-KR-M,19
J0341-DR-M,17
J0230-SKD-S,16
JNE3797-KR-L,16
J0341-DR-L,15


In [17]:
amazon_df[amazon_df['SKU']=='JNE3797-KR-M']['Amount'].unique()

array([ 725.  , 1450.  ,     nan,  771.  , 2175.  ,  690.48,    0.  ,
        715.  , 1430.  ,  680.95,  761.  ,  724.76,  734.29,  734.28,
        735.  ,  724.  ,  689.52,  799.  ,  700.  , 1470.  ])

In [18]:
amazon_df[(amazon_df['SKU']=='JNE3797-KR-M') & (amazon_df['Qty']==1)]['Amount'].unique()

array([725., 771.,   0., 715., 761., 735., 724., 799.])

Now we see that besides the fact that the amount is not unique for every SKU we also have cases when the amount represents the total price of multiple items. So we need to create a new dataframe with the average unit price for each SKU and use that dataframe to fill the missing values. </br>
But we also have 0 values in this column which should be dealt with the same way as the missing values. Therefore, first we'll fill the missing values in the column with 0 and create a new column representing the price of each item.

In [19]:
amazon_df['Amount'].fillna(0, inplace=True)
amazon_df['Price']=amazon_df['Amount']/amazon_df['Qty']

In [20]:
amazon_df['Price'].value_counts(dropna=False)

NaN      7669
399.0    5460
inf      5133
771.0    2805
0.0      2467
         ... 
723.0       1
891.0       1
334.0       1
695.0       1
639.0       1
Name: Price, Length: 749, dtype: int64

In [21]:
amazon_df[amazon_df['Price']==np.inf].head()

,Order ID,Date,Status,Fulfillment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,Amount,ship-city,ship-state,ship-postal-code,promotion-ids,B2B,Price
0,405-8078784-5731545,2022-04-30,Cancelled,Easy Ship,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,Cancelled,0,647.62,MUMBAI,MAHARASHTRA,400081.0,no promotion,False,inf
3,403-9615377-8133951,2022-04-30,Cancelled,Easy Ship,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,Cancelled,0,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,no promotion,False,inf
23,404-6019946-2909948,2022-04-30,Cancelled,Easy Ship,Amazon.in,Standard,SET291,SET291-KR-PP-M,Set,M,B099NK55YG,Cancelled,0,570.48,pune,MAHARASHTRA,411044.0,no promotion,False,inf
83,404-6522553-9345930,2022-04-30,Cancelled,Easy Ship,Amazon.in,Standard,SET377,SET377-KR-NP-M,Set,M,B09TZV23QS,Cancelled,0,1105.36,DEHRADUN,UTTARAKHAND,248001.0,no promotion,False,inf
178,171-1224053-5752314,2022-04-30,Cancelled,Easy Ship,Amazon.in,Standard,J0344,J0344-TP-L,Top,L,B0986XYFFP,Cancelled,0,463.81,BENGALURU,KARNATAKA,560087.0,no promotion,False,inf


In [22]:
amazon_df[amazon_df['Price'].isna()].head()

,Order ID,Date,Status,Fulfillment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,Amount,ship-city,ship-state,ship-postal-code,promotion-ids,B2B,Price
8,407-5443024-5233168,2022-04-30,Cancelled,Amazon,Amazon.in,Expedited,SET200,SET200-KR-NP-A-XXXL,Set,3XL,B08L91ZZXN,Cancelled,0,0.0,HYDERABAD,TELANGANA,500008.0,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN
29,404-5933402-8801952,2022-04-30,Cancelled,Easy Ship,Amazon.in,Standard,JNE2132,JNE2132-KR-398-XXXL,kurta,3XL,B07JG3CND8,Cancelled,0,0.0,GUWAHATI,ASSAM,781003.0,no promotion,False,NaN
65,171-4137548-0481151,2022-04-30,Cancelled,Amazon,Amazon.in,Expedited,JNE3373,JNE3373-KR-XXL,kurta,XXL,B082W8RWN1,Cancelled,0,0.0,Dahod,Gujarat,389151.0,no promotion,False,NaN
84,403-9950518-0349133,2022-04-30,Cancelled,Amazon,Amazon.in,Expedited,JNE3510,JNE3510-KR-M,kurta,M,B08WPR5MCB,Cancelled,0,0.0,HYDERABAD,TELANGANA,500072.0,no promotion,False,NaN
95,405-9112089-3379536,2022-04-30,Cancelled,Amazon,Amazon.in,Expedited,JNE3405,JNE3405-KR-L,kurta,L,B081WSCKPQ,Cancelled,0,0.0,PUNE,MAHARASHTRA,411046.0,no promotion,False,NaN


In [23]:
amazon_df[(amazon_df['Courier Status']=='Cancelled') & (amazon_df['Qty']==0)]['Price'].value_counts(dropna=False)

NaN    7669
inf    5133
Name: Price, dtype: int64

So, now we have different problem: All the cancelled orders have 0 value for quantity, which results around 5k infinity values and 7k null values in our new price column. We can replace those values with 0s and try to find out the right quantity for them after filling the price, simply by dividing the amount by the price.

In [24]:
amazon_df['Price'].replace(np.inf, 0, inplace=True)
amazon_df['Price'].fillna(0, inplace=True)
amazon_df['Price'].value_counts(dropna=False)

0.0       15269
399.0      5460
771.0      2805
735.0      2441
487.0      2300
          ...  
779.0         1
1030.0        1
1670.0        1
891.0         1
639.0         1
Name: Price, Length: 747, dtype: int64

Now let's create the dataframe for the unit prices that we mentioned above.

In [25]:
prices=amazon_df[amazon_df['Qty']==1].groupby('SKU')[['Amount']].mean()
prices.reset_index(inplace=True)
prices.head()

,SKU,Amount
0,AN201-RED-M,229.00
1,AN201-RED-XL,301.00
2,AN201-RED-XXL,229.00
3,AN202-ORANGE-M,265.00
4,AN202-ORANGE-S,285.25


In [26]:
no_prices=amazon_df[amazon_df['Price']==0][['SKU']].drop_duplicates()
no_prices.shape[0]

4064

In [27]:
new_prices=prices.merge(no_prices, how='right', on='SKU')
new_prices.head()

,SKU,Amount
0,SET389-KR-NP-S,635.103448
1,J0341-DR-L,804.549020
2,SET200-KR-NP-A-XXXL,548.833333
3,SET291-KR-PP-M,561.343434
4,JNE2132-KR-398-XXXL,545.000000


In [28]:
new_prices['Price']=new_prices['Amount'].round(2)
new_prices.drop(columns='Amount', inplace=True)
new_prices['Price'].isna().sum()

56

We have 56 sku codes that still doesn't have a price. We'll try different approach to deal with these values later. But for now we'll replace them with 0.

In [29]:
new_prices['Price'].fillna(0, inplace=True)

In [30]:
amazon_df=amazon_df.merge(new_prices, how='left', on='SKU', suffixes=('', '_new'))
amazon_df.head()

,Order ID,Date,Status,Fulfillment,Sales Channel,ship-service-level,Style,SKU,Category,Size,...,Courier Status,Qty,Amount,ship-city,ship-state,ship-postal-code,promotion-ids,B2B,Price,Price_new
0,405-8078784-5731545,2022-04-30,Cancelled,Easy Ship,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,...,Cancelled,0,647.62,MUMBAI,MAHARASHTRA,400081.0,no promotion,False,0.0,635.10
1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Easy Ship,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,...,Shipped,1,406.00,BENGALURU,KARNATAKA,560085.0,Amazon PLCC Free-Financing Universal Merchant ...,False,406.0,412.15
2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,...,Shipped,1,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN Core Free Shipping 2015/04/08 23-48-5-108,True,329.0,332.25
3,403-9615377-8133951,2022-04-30,Cancelled,Easy Ship,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,...,Cancelled,0,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,no promotion,False,0.0,804.55
4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,...,Shipped,1,574.00,CHENNAI,TAMIL NADU,600073.0,no promotion,False,574.0,628.69


Now we'll replace the 0 values in the 'Price' column with our new prices.

In [31]:
amazon_df.loc[amazon_df['Price']==0, 'Price']=amazon_df['Price_new']
amazon_df.drop(columns='Price_new', inplace=True)

Now let's deal with the remaining 0 values in 'Price' and 'Qty' columns.

In [32]:
amazon_df[amazon_df['Price']==0][['Qty', 'Amount', 'Price']].head(10).T

,363,3630,3632,3635,3969,4033,6742,6791,6800,6830
Qty,0.0,1.0,1.0,2.0,0.0,0.00,0.0,0.0,0.0,0.0
Amount,0.0,0.0,0.0,0.0,0.0,493.34,0.0,0.0,0.0,0.0
Price,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0


In [33]:
amazon_df[(amazon_df['Price']==0) & (amazon_df['Amount']!=0)]['Qty'].value_counts()

0    28
Name: Qty, dtype: int64

In [34]:
amazon_df[(amazon_df['Price']==0) & (amazon_df['Qty']!=0)]['Amount'].value_counts()

0.0    31
Name: Amount, dtype: int64

We can see that we have cases when we have the total amount but don't have the quantity and vice versa. And we'll not be able to use the rows with no price values even when we know the quantity. Therefore we'll drop all the rows with 0 'Price' and 'Amount' values. And for the rows with 0 'Price' and 'Qty' column we'll replace the 'Price' column with the value in 'Amount' and change 'Qty' to 1. Also we'll find the quantity for those rows where we have the price and amount information.

In [35]:
mask_1=amazon_df[(amazon_df['Price']==0) & (amazon_df['Amount']==0)].index
amazon_df.drop(index=mask_1, inplace=True)

mask_2=amazon_df[(amazon_df['Price']==0) & (amazon_df['Qty']==0)].index
amazon_df.loc[mask_2, 'Price']=amazon_df['Amount']
amazon_df.loc[mask_2, 'Qty']=1

mask_3=amazon_df[(amazon_df['Price']!=0) & (amazon_df['Amount']!=0) & (amazon_df['Qty']==0)].index
amazon_df.loc[mask_3, 'Qty']=round(amazon_df['Amount']/amazon_df['Price'])

Let's check the 0 values in those columns after the transformations above.

In [36]:
for col in ['Price', 'Amount', 'Qty']:
    val_0=amazon_df[amazon_df[col]==0].shape[0]
    print("There are {} '0' values in the {} column".format(val_0, col))

There are 0 '0' values in the Price column
There are 10067 '0' values in the Amount column
There are 7631 '0' values in the Qty column


Now we can replace 0 values in the 'Amount' column with the values from the 'Price' column and repeat the step of finding the quantity for them.

In [37]:
mask_4=amazon_df[(amazon_df['Amount']==0) & (amazon_df['Qty']==0)].index
amazon_df.loc[mask_4, 'Amount']=amazon_df['Price']

mask_5=amazon_df[(amazon_df['Amount']==0) & (amazon_df['Qty']!=0)].index
amazon_df.loc[mask_5, 'Amount']=amazon_df['Price']*amazon_df['Qty']

mask_6=amazon_df[amazon_df['Qty']==0].index
amazon_df.loc[mask_6, 'Qty']=round(amazon_df['Amount']/amazon_df['Price'])

In [38]:
for col in ['Price', 'Amount', 'Qty']:
    val_0=amazon_df[amazon_df[col]==0].shape[0]
    print("There are {} '0' values in the {} column".format(val_0, col))

There are 0 '0' values in the Price column
There are 0 '0' values in the Amount column
There are 0 '0' values in the Qty column


In [39]:
amazon_df.head()

,Order ID,Date,Status,Fulfillment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,Amount,ship-city,ship-state,ship-postal-code,promotion-ids,B2B,Price
0,405-8078784-5731545,2022-04-30,Cancelled,Easy Ship,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,Cancelled,1,647.62,MUMBAI,MAHARASHTRA,400081.0,no promotion,False,635.10
1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Easy Ship,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,406.00,BENGALURU,KARNATAKA,560085.0,Amazon PLCC Free-Financing Universal Merchant ...,False,406.00
2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN Core Free Shipping 2015/04/08 23-48-5-108,True,329.00
3,403-9615377-8133951,2022-04-30,Cancelled,Easy Ship,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,Cancelled,1,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,no promotion,False,804.55
4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,574.00,CHENNAI,TAMIL NADU,600073.0,no promotion,False,574.00


Due to usage of average prices for some SKU codes now we have slightly different values in 'Amount' and 'Price' columns. We can use the 'Qty' and 'Amount' columns to correct the values in 'Price' column.

In [40]:
amazon_df['Price']=amazon_df['Amount']/amazon_df['Qty']

In [41]:
amazon_df.head()

,Order ID,Date,Status,Fulfillment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,Amount,ship-city,ship-state,ship-postal-code,promotion-ids,B2B,Price
0,405-8078784-5731545,2022-04-30,Cancelled,Easy Ship,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,Cancelled,1,647.62,MUMBAI,MAHARASHTRA,400081.0,no promotion,False,647.62
1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Easy Ship,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,406.00,BENGALURU,KARNATAKA,560085.0,Amazon PLCC Free-Financing Universal Merchant ...,False,406.00
2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN Core Free Shipping 2015/04/08 23-48-5-108,True,329.00
3,403-9615377-8133951,2022-04-30,Cancelled,Easy Ship,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,Cancelled,1,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,no promotion,False,753.33
4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,574.00,CHENNAI,TAMIL NADU,600073.0,no promotion,False,574.00


And we dealt with all the 0 values in those columns. Now let's check how our dataset looks in terms of missing values and column data types.

In [42]:
amazon_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128873 entries, 0 to 128941
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Order ID            128873 non-null  object        
 1   Date                128873 non-null  datetime64[ns]
 2   Status              128873 non-null  object        
 3   Fulfillment         128873 non-null  object        
 4   Sales Channel       128873 non-null  object        
 5   ship-service-level  128873 non-null  object        
 6   Style               128873 non-null  object        
 7   SKU                 128873 non-null  object        
 8   Category            128873 non-null  object        
 9   Size                128873 non-null  object        
 10  ASIN                128873 non-null  object        
 11  Courier Status      128873 non-null  object        
 12  Qty                 128873 non-null  int64         
 13  Amount              128873 no

In [43]:
amazon_df.describe()

,Qty,Amount,ship-postal-code,Price
count,128873.000000,128873.000000,128873.000000,128873.000000
mean,1.003957,660.153552,463977.031861,657.854400
std,0.091289,268.864882,191484.982306,263.959146
min,1.000000,124.500000,110001.000000,124.500000
25%,1.000000,458.000000,382421.000000,458.000000
50%,1.000000,614.000000,500033.000000,613.000000
75%,1.000000,788.000000,600024.000000,788.000000
max,15.000000,5685.000000,989898.000000,2598.000000


Everything looks fine besides the data type and format of 'ship-postal-code' which should be categorical. And it would be better if the price column was next to the quantity column.

In [44]:
amazon_df['ship-postal-code']=amazon_df['ship-postal-code'].astype('int')
amazon_df['ship-postal-code']=amazon_df['ship-postal-code'].astype('object')
amazon_df.columns

Index(['Order ID', 'Date', 'Status', 'Fulfillment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'Amount', 'ship-city', 'ship-state',
       'ship-postal-code', 'promotion-ids', 'B2B', 'Price'],
      dtype='object')

In [45]:
new_cols=['Order ID', 'Date', 'Status', 'Fulfillment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Price', 'Qty', 'Amount', 'ship-city', 'ship-state',
       'ship-postal-code', 'promotion-ids', 'B2B']
amazon_df=amazon_df.reindex(columns=new_cols)
amazon_df.head()

,Order ID,Date,Status,Fulfillment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Price,Qty,Amount,ship-city,ship-state,ship-postal-code,promotion-ids,B2B
0,405-8078784-5731545,2022-04-30,Cancelled,Easy Ship,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,Cancelled,647.62,1,647.62,MUMBAI,MAHARASHTRA,400081,no promotion,False
1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Easy Ship,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,406.00,1,406.00,BENGALURU,KARNATAKA,560085,Amazon PLCC Free-Financing Universal Merchant ...,False
2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,329.00,1,329.00,NAVI MUMBAI,MAHARASHTRA,410210,IN Core Free Shipping 2015/04/08 23-48-5-108,True
3,403-9615377-8133951,2022-04-30,Cancelled,Easy Ship,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,Cancelled,753.33,1,753.33,PUDUCHERRY,PUDUCHERRY,605008,no promotion,False
4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,574.00,1,574.00,CHENNAI,TAMIL NADU,600073,no promotion,False


And the last step in data wrangling for this dataset would be chechking for duplicates.

In [46]:
amazon_df.duplicated().sum()

6

In [47]:
amazon_df.drop_duplicates(inplace=True)

So we had 6 duplicate rows which were dropped.

## International sales

Now let's have a look at our second dataset which represents the international sales of the e-commerce company.

In [48]:
inter_df=pd.read_csv('Original datasets/International sale Report.csv')
inter_df.head()

,index,DATE,Months,CUSTOMER,Style,SKU,Size,PCS,RATE,GROSS AMT
0,0,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-L,L,1.00,616.56,617.00
1,1,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-XL,XL,1.00,616.56,617.00
2,2,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-XXL,XXL,1.00,616.56,617.00
3,3,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5009,MEN5009-KR-L,L,1.00,616.56,617.00
4,4,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5011,MEN5011-KR-L,L,1.00,616.56,617.00


In [49]:
inter_df.drop(columns='index', inplace=True)
inter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37432 entries, 0 to 37431
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   DATE       37431 non-null  object
 1   Months     37407 non-null  object
 2   CUSTOMER   36392 non-null  object
 3   Style      36392 non-null  object
 4   SKU        34958 non-null  object
 5   Size       36392 non-null  object
 6   PCS        36392 non-null  object
 7   RATE       36392 non-null  object
 8   GROSS AMT  36392 non-null  object
dtypes: object(9)
memory usage: 2.6+ MB


We have the same amount of null values in 'CUSTOMER', 'Style', 'Size', 'PCS', 'RATE' and 'GROSS AMT' columns. Let's take a look at those rows.

In [50]:
inter_df[inter_df['CUSTOMER'].isna()][['Style', 'Size', 'PCS', 'RATE', 'GROSS AMT']].value_counts(dropna=False)

Style  Size  PCS  RATE  GROSS AMT
NaN    NaN   NaN  NaN   NaN          1040
dtype: int64

As we can see those rows are missing most of the information, so we can drop them.

In [51]:
mask_11=inter_df[inter_df['CUSTOMER'].isna()].index
inter_df.drop(index=mask_11, inplace=True)
inter_df.reset_index(inplace=True, drop=True)
inter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36392 entries, 0 to 36391
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   DATE       36392 non-null  object
 1   Months     36392 non-null  object
 2   CUSTOMER   36392 non-null  object
 3   Style      36392 non-null  object
 4   SKU        34958 non-null  object
 5   Size       36392 non-null  object
 6   PCS        36392 non-null  object
 7   RATE       36392 non-null  object
 8   GROSS AMT  36392 non-null  object
dtypes: object(9)
memory usage: 2.5+ MB


Now if we take a look at our values in our columns we'll see that a lot of values are switched up.

In [52]:
mask_11=inter_df['DATE'].str.contains('\d{2}\-\d{2}\-\d{2}', regex=True)
wrong_date=inter_df[mask_11==False]
wrong_date.head(10)

,DATE,Months,CUSTOMER,Style,SKU,Size,PCS,RATE,GROSS AMT
18635,CUSTOMER,DATE,Months,Style,SKU,PCS,RATE,GROSS AMT,Stock
18636,REVATHY LOGANATHAN,06-05-21,Jun-21,MEN5004,MEN5004-KR-L,1.00,616.56,617.00,8.00
18637,REVATHY LOGANATHAN,06-05-21,Jun-21,MEN5004,MEN5004-KR-XL,1.00,616.56,617.00,4.00
18638,REVATHY LOGANATHAN,06-05-21,Jun-21,MEN5004,MEN5004-KR-XXL,1.00,616.56,617.00,6.00
18639,REVATHY LOGANATHAN,06-05-21,Jun-21,MEN5009,MEN5009-KR-L,1.00,616.56,617.00,6.00
18640,REVATHY LOGANATHAN,06-05-21,Jun-21,MEN5011,MEN5011-KR-L,1.00,616.56,617.00,11.00
18641,REVATHY LOGANATHAN,06-05-21,Jun-21,MEN5025,MEN5025-KR-L,1.00,649.03,649.00,6.00
18642,REVATHY LOGANATHAN,06-05-21,Jun-21,MEN5015,MEN5015-KR-XL,1.00,616.56,617.00,15.00
18643,REVATHY LOGANATHAN,06-05-21,Jun-21,MEN5022,MEN5022-KR-XXL,1.00,649.03,649.00,48.00
18644,REVATHY LOGANATHAN,06-05-21,Jun-21,MEN5014,MEN5014-KR-S,1.00,649.03,649.00,30.00


It appears that starting from row 19675 we have different order of columns and instead of 'size' we have 'stock' column. Let's divide our data into two datasets and bring to the same format and order.

In [53]:
inter_2=inter_df.iloc[18635:].copy()
inter_2.head()

,DATE,Months,CUSTOMER,Style,SKU,Size,PCS,RATE,GROSS AMT
18635,CUSTOMER,DATE,Months,Style,SKU,PCS,RATE,GROSS AMT,Stock
18636,REVATHY LOGANATHAN,06-05-21,Jun-21,MEN5004,MEN5004-KR-L,1.00,616.56,617.00,8.00
18637,REVATHY LOGANATHAN,06-05-21,Jun-21,MEN5004,MEN5004-KR-XL,1.00,616.56,617.00,4.00
18638,REVATHY LOGANATHAN,06-05-21,Jun-21,MEN5004,MEN5004-KR-XXL,1.00,616.56,617.00,6.00
18639,REVATHY LOGANATHAN,06-05-21,Jun-21,MEN5009,MEN5009-KR-L,1.00,616.56,617.00,6.00


In [54]:
inter_2.tail()

,DATE,Months,CUSTOMER,Style,SKU,Size,PCS,RATE,GROSS AMT
36387,AVIN,03-31-22,Mar-22,PJNE3423,PJNE3423-KR-4XL,1.00,537.50,538.00,4.00
36388,AVIN,03-31-22,Mar-22,PJNE3404,PJNE3404-KR-4XL,1.00,500.00,500.00,5.00
36389,AVIN,03-31-22,Mar-22,PJNE3423,PJNE3423-KR-4XL,1.00,537.50,538.00,4.00
36390,AVIN,03-31-22,Mar-22,SET290,SET290-KR-DPT-M,1.00,812.50,812.00,7.00
36391,AVIN,03-31-22,Mar-22,SHIPPING,SHIPPING,1.00,3666.00,3666.00,0.00


After comparing the first lines of the new dataset with the first lines of the original dataset we can see that the information in all the mutual columns is the same, but we we'll check this statement after joining these two parts of datasets into a new one.

In [55]:
new_cols=inter_2.loc[18635].values
print(new_cols)

['CUSTOMER' 'DATE' 'Months' 'Style' 'SKU' 'PCS' 'RATE' 'GROSS AMT' 'Stock']


In [56]:
inter_2.columns=new_cols
inter_2.drop(index=18635, inplace=True)
inter_2.head()

,CUSTOMER,DATE,Months,Style,SKU,PCS,RATE,GROSS AMT,Stock
18636,REVATHY LOGANATHAN,06-05-21,Jun-21,MEN5004,MEN5004-KR-L,1.00,616.56,617.00,8.00
18637,REVATHY LOGANATHAN,06-05-21,Jun-21,MEN5004,MEN5004-KR-XL,1.00,616.56,617.00,4.00
18638,REVATHY LOGANATHAN,06-05-21,Jun-21,MEN5004,MEN5004-KR-XXL,1.00,616.56,617.00,6.00
18639,REVATHY LOGANATHAN,06-05-21,Jun-21,MEN5009,MEN5009-KR-L,1.00,616.56,617.00,6.00
18640,REVATHY LOGANATHAN,06-05-21,Jun-21,MEN5011,MEN5011-KR-L,1.00,616.56,617.00,11.00


In [57]:
new_cols2=['DATE', 'Months', 'CUSTOMER', 'Style', 'SKU', 'Size', 'PCS', 'RATE', 'GROSS AMT', 'Stock']

inter_1=inter_df.iloc[:18635].copy()
inter_1['Stock']=0
inter_2['Size']='?'

inter_2=inter_2.reindex(columns=new_cols2)

In [58]:
inter_1.head()

,DATE,Months,CUSTOMER,Style,SKU,Size,PCS,RATE,GROSS AMT,Stock
0,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-L,L,1.00,616.56,617.00,0
1,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-XL,XL,1.00,616.56,617.00,0
2,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-XXL,XXL,1.00,616.56,617.00,0
3,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5009,MEN5009-KR-L,L,1.00,616.56,617.00,0
4,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5011,MEN5011-KR-L,L,1.00,616.56,617.00,0


In [59]:
stock_info=inter_2[['CUSTOMER', 'SKU', 'Stock']]
stock_info=stock_info.drop_duplicates()
stock_info.head()

,CUSTOMER,SKU,Stock
18636,REVATHY LOGANATHAN,MEN5004-KR-L,8.00
18637,REVATHY LOGANATHAN,MEN5004-KR-XL,4.00
18638,REVATHY LOGANATHAN,MEN5004-KR-XXL,6.00
18639,REVATHY LOGANATHAN,MEN5009-KR-L,6.00
18640,REVATHY LOGANATHAN,MEN5011-KR-L,11.00


In [60]:
new_inter_df=pd.concat([inter_1, inter_2], axis=0)
new_inter_df.head()

,DATE,Months,CUSTOMER,Style,SKU,Size,PCS,RATE,GROSS AMT,Stock
0,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-L,L,1.00,616.56,617.00,0
1,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-XL,XL,1.00,616.56,617.00,0
2,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-XXL,XXL,1.00,616.56,617.00,0
3,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5009,MEN5009-KR-L,L,1.00,616.56,617.00,0
4,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5011,MEN5011-KR-L,L,1.00,616.56,617.00,0


Now that we have all the information in the right columns let's check our statement for duplicate rows.

In [61]:
col_subset=['CUSTOMER', 'DATE', 'Months', 'Style', 'SKU', 'PCS', 'RATE', 'GROSS AMT']
new_inter_df.duplicated(subset=col_subset).sum()

24572

It seems like all of the rows were duplicated, some of them more than once. Let's drop the duplicates.

In [62]:
new_inter_df.drop_duplicates(subset=col_subset, inplace=True, keep='first')

In [63]:
new_inter_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11819 entries, 0 to 36391
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   DATE       11819 non-null  object
 1   Months     11819 non-null  object
 2   CUSTOMER   11819 non-null  object
 3   Style      11819 non-null  object
 4   SKU        11238 non-null  object
 5   Size       11819 non-null  object
 6   PCS        11819 non-null  object
 7   RATE       11819 non-null  object
 8   GROSS AMT  11819 non-null  object
 9   Stock      11819 non-null  object
dtypes: object(10)
memory usage: 1015.7+ KB


Now let's try to convert the data types for 'DATE', 'PCS', 'RATE' and 'GROSS AMT' columns.

In [64]:
new_inter_df['DATE']=pd.to_datetime(new_inter_df['DATE'])

float_cols=['PCS', 'RATE', 'GROSS AMT']
for col in float_cols:
    new_inter_df[col]=new_inter_df[col].astype('float')
    
new_inter_df['PCS']=new_inter_df['PCS'].astype('int64')

Now let's have a look at the 'stock' and 'Size' columns.

In [65]:
new_inter_df['Size'].value_counts()

L           2287
M           2082
XL          2017
XXL         1643
S           1604
XXXL        1052
XS           543
?            295
Free         118
4XL           59
5XL           53
6XL           46
S TO XXL       9
FREE           6
s              5
Name: Size, dtype: int64

We have 295 rows with no size value. But hopefully we can extract the size information from the 'SKU' column.

In [66]:
new_inter_df[new_inter_df['Size']=='?'].head()

,DATE,Months,CUSTOMER,Style,SKU,Size,PCS,RATE,GROSS AMT,Stock
18656,2021-06-05,Jun-21,REVATHY LOGANATHAN,SHIPPING,SHIPPING,?,1,2250.0,2250.0,0.00
18711,2021-06-08,Jun-21,FARIA ESSOPP,SHIPPING,SHIPPING,?,1,11000.0,11000.0,0.00
18776,2021-06-11,Jun-21,MANGALAM SHOP,SHIPPING,SHIPPING,?,1,8600.0,8600.0,0.00
18805,2021-06-15,Jun-21,THANA NAGISSWARY L MARIMUTHU,SHIPPING,SHIPPING,?,1,5800.0,5800.0,0.00
18825,2021-06-17,Jun-21,REVATHY LOGANATHAN,SHIPPING,SHIPPING,?,1,2200.0,2200.0,0.00


It seems like we have the shipping costs for the orders in these rows. Let's double check if all the rows are shipping costs.

In [67]:
new_inter_df[new_inter_df['Size']=='?'][['Style', 'SKU']].value_counts()

Style               SKU               
SHIPPING            SHIPPING              218
CMB5                CMB5-JNE3849            5
                    CMB5-JNE3828            5
                    CMB5-JNE3853            5
                    CMB5-JNE3844            4
                    CMB5-JNE3855            4
                    CMB5-JNE3827            4
                    CMB5-JNE3848            4
                    CMB5-JNE3826            4
                    CMB5-JNE3840            3
                    CMB5-JNE3845            3
TAGS                TAGS                    3
CMB5                CMB5-JNE3839            3
                    CMB5-JNE3830            3
                    CMB5-JNE3852            3
                    CMB5-JNE3831            2
SHIPPING CHARGES    SHIPPING CHARGES        2
LABEL CHARGE        LABEL CHARGE            2
CMB5                CMB5-JNE3858            2
TAGS(LABOUR)        TAGS(LABOUR)            2
CMB5                CMB5-JNE3851         

So, most of them are shipping costs, but there are some other rows as well. We have an item with the 'CMB5' value in 'Style' column and few other expenses which we can drop. We also need to save the shipping costs in a separate dataset.

In [68]:
new_inter_df.loc[new_inter_df['Style']=='CMB5', 'Size']='Free'

mask_12=new_inter_df['Style'].isin(['SHIPPING', 'SHIPPING CHARGES'])
ship_costs=new_inter_df.loc[mask_12, ['DATE', 'CUSTOMER', 'RATE']]
ship_costs.head()

,DATE,CUSTOMER,RATE
18656,2021-06-05,REVATHY LOGANATHAN,2250.0
18711,2021-06-08,FARIA ESSOPP,11000.0
18776,2021-06-11,MANGALAM SHOP,8600.0
18805,2021-06-15,THANA NAGISSWARY L MARIMUTHU,5800.0
18825,2021-06-17,REVATHY LOGANATHAN,2200.0


In [69]:
drop_mask=new_inter_df[new_inter_df['Size']=='?'].index
new_inter_df.drop(index=drop_mask, inplace=True)

In [70]:
new_inter_df['Size'].value_counts()

L           2287
M           2082
XL          2017
XXL         1643
S           1604
XXXL        1052
XS           543
Free         184
4XL           59
5XL           53
6XL           46
S TO XXL       9
FREE           6
s              5
Name: Size, dtype: int64

In [71]:
new_inter_df['Size'].replace('FREE', 'Free', inplace=True)
new_inter_df['Size'].replace('s', 'S', inplace=True)

Now let's have a look at our 'stock' column.

In [72]:
new_inter_df['Stock'].value_counts()

0       11524
0.00       66
Name: Stock, dtype: int64

So we've dropped all the rows with the information of stock items, but we saved them in a separate dataset and now we can merge them with the_inter_df data.

In [73]:
int_df=pd.merge(new_inter_df, stock_info, how='left', on=['CUSTOMER', 'SKU'])
int_df.head()

,DATE,Months,CUSTOMER,Style,SKU,Size,PCS,RATE,GROSS AMT,Stock_x,Stock_y
0,2021-06-05,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-L,L,1,616.56,617.0,0,8.00
1,2021-06-05,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-XL,XL,1,616.56,617.0,0,4.00
2,2021-06-05,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-XXL,XXL,1,616.56,617.0,0,6.00
3,2021-06-05,Jun-21,REVATHY LOGANATHAN,MEN5009,MEN5009-KR-L,L,1,616.56,617.0,0,6.00
4,2021-06-05,Jun-21,REVATHY LOGANATHAN,MEN5011,MEN5011-KR-L,L,1,616.56,617.0,0,11.00


In [74]:
int_df.shape

(11590, 11)

In [75]:
int_df.drop(columns='Stock_x', inplace=True)
int_df.rename(columns={'Stock_y': 'Stock'}, inplace=True)

In [76]:
int_df['Stock'].value_counts(dropna=False)

0.00      1032
4.00       904
5.00       656
NaN        575
3.00       569
          ... 
407.00       1
149.00       1
357.00       1
170.00       1
325.00       1
Name: Stock, Length: 253, dtype: int64

We still have all of 0 values but now we also have null values which we can replace with 0.

In [77]:
int_df['Stock'].fillna(0, inplace=True)

Now let's join the Shipping information to our dataset.

In [82]:
int_df=pd.merge(int_df, ship_costs, how='left', on=['DATE', 'CUSTOMER'])
int_df.head()

,DATE,Months,CUSTOMER,Style,SKU,Size,PCS,RATE_x,GROSS AMT,Stock,RATE_y
0,2021-06-05,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-L,L,1,616.56,617.0,8.00,2250.0
1,2021-06-05,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-XL,XL,1,616.56,617.0,4.00,2250.0
2,2021-06-05,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-XXL,XXL,1,616.56,617.0,6.00,2250.0
3,2021-06-05,Jun-21,REVATHY LOGANATHAN,MEN5009,MEN5009-KR-L,L,1,616.56,617.0,6.00,2250.0
4,2021-06-05,Jun-21,REVATHY LOGANATHAN,MEN5011,MEN5011-KR-L,L,1,616.56,617.0,11.00,2250.0


In [83]:
int_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12775 entries, 0 to 12774
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   DATE       12775 non-null  datetime64[ns]
 1   Months     12775 non-null  object        
 2   CUSTOMER   12775 non-null  object        
 3   Style      12775 non-null  object        
 4   SKU        12193 non-null  object        
 5   Size       12775 non-null  object        
 6   PCS        12775 non-null  int64         
 7   RATE_x     12775 non-null  float64       
 8   GROSS AMT  12775 non-null  float64       
 9   Stock      12775 non-null  object        
 10  RATE_y     11430 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(6)
memory usage: 1.2+ MB


Let's change some of the column names of the data and take a look at the null values in 'SKU' column.

In [84]:
int_df.columns=['Date', 'Month', 'Customer_name', 'Style', 'SKU', 'Size', 'Quantity', 'Price', 'Total', 'Stock', 'Shipping(per order)']

In [85]:
int_df[int_df['SKU'].isna()]

,Date,Month,Customer_name,Style,SKU,Size,Quantity,Price,Total,Stock,Shipping(per order)
2787,2021-08-25,Aug-21,BINCY SKARIA,JNE1234,NaN,Free,1,409.00,401.0,0.00,5528.0
2789,2021-08-25,Aug-21,BINCY SKARIA,JNE1408,NaN,Free,1,469.00,460.0,0.00,5528.0
4474,2021-09-15,Sep-21,MANISH DHOORUNDHUR,JNE1408,NaN,Free,1,469.00,469.0,0.00,5120.0
7585,2021-12-13,Dec-21,MR. GOPALDAS,JNE1233,NaN,Free,1,513.00,513.0,0.00,5439.0
8888,2022-02-05,Feb-22,THILAS BOMBAY BOUTIQUE SDN BHD,JNE1408,NaN,Free,1,503.75,489.0,0.00,4200.0
...,...,...,...,...,...,...,...,...,...,...,...
12696,2022-04-30,Apr-22,Rino Sandaran,J0008,NaN,S,1,1275.00,1275.0,0,NaN
12697,2022-04-30,Apr-22,Rino Sandaran,J0280,NaN,S,1,1812.50,1812.5,0,NaN
12698,2022-04-30,Apr-22,Rino Sandaran,SET350,NaN,M,1,1512.50,1512.5,0,NaN
12699,2022-04-30,Apr-22,Rino Sandaran,J0242,NaN,XXXL,1,1312.50,1312.5,0,NaN


We can simply copy the information from the 'Style' column for these rows, because the SKU contains the 'Style' information.

In [87]:
int_df['SKU'].fillna(int_df['Style'], inplace=True)

And for the null values in 'Shipping' column we can impute them with 0.

In [91]:
int_df['Shipping(per order)'].fillna(0, inplace=True)
int_df['Stock']=int_df['Stock'].astype('float64')

In [92]:
int_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12775 entries, 0 to 12774
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date                 12775 non-null  datetime64[ns]
 1   Month                12775 non-null  object        
 2   Customer_name        12775 non-null  object        
 3   Style                12775 non-null  object        
 4   SKU                  12775 non-null  object        
 5   Size                 12775 non-null  object        
 6   Quantity             12775 non-null  int64         
 7   Price                12775 non-null  float64       
 8   Total                12775 non-null  float64       
 9   Stock                12775 non-null  float64       
 10  Shipping(per order)  12775 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(5)
memory usage: 1.2+ MB


And now we have both of our datasets cleaned and ready for the next part of the project.

In [93]:
amazon_df.to_csv('amazon_sales_1.csv', index=False)
int_df.to_csv('international_sales_1.csv', index=False)